The intent of this exercise is to practice connecting to an API for data extraction using Python, and storing the information in a SQL database. 

Washed Media is a small to medium sized media company out of Austin, TX that creates a variety of podcasts published on Spotify along with other streaming apps. Let's assume they wanted to take a look at thier user engagement after each episode. There could be a variety of ways to do this, but a more user friendly way would be to import the different information sources from places like Spotify, Twitter, Instagram, Reddit etc and visualize the information in Power BI. That would allow for the data to be utilized by members of the company that do not have Python skills. 

One holding place for the different information sources would be a SQL database, then business intelligence applications like Power BI could connect directly to the SQL database to access the different sources of data. In order to have the different data sources compiled into a SQL database, I am setting up just one of a few data pipelines that would be necessary. This data pipeline uses Python to connect to the Spotify API. Granted, this is using my personal spotify account to access the data, so the actual data will be limited, but the general flow would be similar for a podcast owner account. 

In [2]:
pip install spotipy

Note: you may need to restart the kernel to use updated packages.


In [3]:
import spotipy

There is an important step here that has been omitted. I established a connection with the Spotify API, but it used a private token that should not be shared publicly, so I have removed it from this demonstration. 

Washed Media uses a different publisher for every podcast so this next part became a bit tedious. 

I need information about the following podcasts from Washed Media that are posted on Spotify:

- Circling Back
- Too Much Dip
- The Mail-In Podcast
- Sunday Scaries
- Club Cool
- Brunch
- Cold Stove
- Outgoing Without Going Out
- Internet Party

It is important to note that the following podcasts are not available on spotify, but are an important part of the Washed Media portfolio and will not be apart of this analysis.

- Happy Hour Live
- Spooky Szn
- The Worst Of

Collect Washed Media Podcasts IDs

In [5]:
show_names = ['Circling Back', 'Too Much Dip', 'The Mail-In Podcast', 'The Sunday Scaries Podcast', 'Club Cool', 'Brunch', 'Cold Stove', 'Outgoing Without Going Out', 'Internet Party']
show_name = []
show_id = []
show_publisher = []
for name in show_names:
    for i in range(0, 10):
        shows_results = sp.search(q= name,type = 'show', market = 'US', limit=50,offset=i)
        for i, t in enumerate(shows_results['shows']['items']):
            show_name.append(t['name'])
            show_id.append(t['id'])
            show_publisher.append(t['publisher'])

In [6]:
import pandas as pd
show_dataframe = pd.DataFrame({'show' : show_name, 'id' : show_id, 'publisher' : show_publisher})
print(show_dataframe.shape)
show_dataframe.head()

(3074, 3)


,show,id,publisher
0,Circling Back,6GWLSnyJKGMDIWsYC0RBG2,Circling Back
1,Let's Circle Back,6nnw4nqKoBPWiEnPTuTg26,Let's Circle Back
2,Circles Off - Sports Betting Podcast,4DxhjavM1fAdYkVw2iJI9f,Betstamp
3,Circle Back,1TSYSNROvPh4sfpM4aV2aY,Nashville Entrepreneur Center
4,Alohomora!: Full Circle,6YE8MhfNAYX2pa0N3FNr5H,Alohomora!


In [7]:
show_dataframe = show_dataframe[show_dataframe['show'].isin(show_names)]

In [8]:
show_dataframe

,show,id,publisher
0,Circling Back,6GWLSnyJKGMDIWsYC0RBG2,Circling Back
500,Too Much Dip,5nVfEJ9bPUvqN2ml5sUGG2,Washed Media
575,The Mail-In Podcast,2v9N3b9tVvkX06NAYwzrvU,The Mail-In
1075,The Sunday Scaries Podcast,0UgblNURXpPYzoBrPHJyi1,The Sunday Scaries Podcast
1572,Club Cool,7sH9LDMFdARFSHcZGE3R34,Club Cool
2073,Brunch,44Cx2rZvJZvKnUlC4AU2lK,BRUNCH
2074,Brunch,2cipMvHP1DnpauVtej7ZOa,Vol Radio
2076,Brunch,36vU4yC3U19CV0L9cVvZvO,Brunch
2122,Brunch,44Cx2rZvJZvKnUlC4AU2lK,BRUNCH
2123,Brunch,2cipMvHP1DnpauVtej7ZOa,Vol Radio


In [9]:
show_dataframe = show_dataframe[show_dataframe['id'].isin(['6GWLSnyJKGMDIWsYC0RBG2', '5nVfEJ9bPUvqN2ml5sUGG2', '2v9N3b9tVvkX06NAYwzrvU', '0UgblNURXpPYzoBrPHJyi1', '7sH9LDMFdARFSHcZGE3R34', '44Cx2rZvJZvKnUlC4AU2lK', '7L8xnywAsJjuQcSk8PbDRc', '20REXE78U9IC7svf8dTrHU'])]

Just for reference, the id of the podcast appears to also be viewable if you copy and paste a podcast's URL into the search bar. It will be apart of the URL. This is how I figured out which Brunch podcast was created by Washed Media. 

In [10]:
show_dataframe = show_dataframe.drop_duplicates(keep = 'first')

In [11]:
show_dataframe

,show,id,publisher
0,Circling Back,6GWLSnyJKGMDIWsYC0RBG2,Circling Back
500,Too Much Dip,5nVfEJ9bPUvqN2ml5sUGG2,Washed Media
575,The Mail-In Podcast,2v9N3b9tVvkX06NAYwzrvU,The Mail-In
1075,The Sunday Scaries Podcast,0UgblNURXpPYzoBrPHJyi1,The Sunday Scaries Podcast
1572,Club Cool,7sH9LDMFdARFSHcZGE3R34,Club Cool
2073,Brunch,44Cx2rZvJZvKnUlC4AU2lK,BRUNCH
2573,Outgoing Without Going Out,7L8xnywAsJjuQcSk8PbDRc,Gabi & Lexi Fuller
2574,Internet Party,20REXE78U9IC7svf8dTrHU,Internet Party


Now that I have identified the exact podcast IDs, I am going to collect all episodes associated with each podcast.

In [12]:
import requests 

show_id = ['6GWLSnyJKGMDIWsYC0RBG2', '5nVfEJ9bPUvqN2ml5sUGG2', '2v9N3b9tVvkX06NAYwzrvU', '0UgblNURXpPYzoBrPHJyi1', '7sH9LDMFdARFSHcZGE3R34', '44Cx2rZvJZvKnUlC4AU2lK', '7L8xnywAsJjuQcSk8PbDRc', '20REXE78U9IC7svf8dTrHU']

for show in show_id:
    path = "https://api.spotify.com/v1/shows/" + show + "episodes"
    
    

In [13]:
show_ids = ['6GWLSnyJKGMDIWsYC0RBG2', '5nVfEJ9bPUvqN2ml5sUGG2', '2v9N3b9tVvkX06NAYwzrvU', '0UgblNURXpPYzoBrPHJyi1', '7sH9LDMFdARFSHcZGE3R34', '44Cx2rZvJZvKnUlC4AU2lK', '7L8xnywAsJjuQcSk8PbDRc', '20REXE78U9IC7svf8dTrHU']
show_dict = {"6GWLSnyJKGMDIWsYC0RBG2": "Circling Back", "5nVfEJ9bPUvqN2ml5sUGG2": "Too Much Dip", "2v9N3b9tVvkX06NAYwzrvU": "Mail-In", "0UgblNURXpPYzoBrPHJyi1" : "Sunday Scaries", '7sH9LDMFdARFSHcZGE3R34': 'Club Cool', "44Cx2rZvJZvKnUlC4AU2lK": "Brunch", '7L8xnywAsJjuQcSk8PbDRc': "Outgoing Without Going Out", '20REXE78U9IC7svf8dTrHU': "Internet Party"}
epi_desc = []
epi_rel_date = []
epi_duration = []
epi_resume = []
show_url = []
show_results = []

for show_id in show_ids:
    url = []
    
    for i in range(0,300,50):
        results = sp.show_episodes(show_id, market = 'US', limit=50,offset=i)
        for j in results:
            url.append(results["href"])
        for i, t in enumerate(results['items']):
            show_url.append(url[0])
            epi_desc.append(t['description'])
            epi_rel_date.append(t['release_date'])
            epi_duration.append(t["duration_ms"])
        

In [14]:
results

{'href': 'https://api.spotify.com/v1/shows/20REXE78U9IC7svf8dTrHU/episodes?offset=250&limit=50&market=US',
 'items': [],
 'limit': 50,
 'next': None,
 'offset': 250,
 'previous': 'https://api.spotify.com/v1/shows/20REXE78U9IC7svf8dTrHU/episodes?offset=200&limit=50&market=US',
 'total': 108}

In [15]:
epi_rel_date

['2022-05-25',
 '2022-05-23',
 '2022-05-18',
 '2022-05-16',
 '2022-05-13',
 '2022-05-11',
 '2022-05-09',
 '2022-05-04',
 '2022-05-02',
 '2022-04-27',
 '2022-04-25',
 '2022-04-20',
 '2022-04-18',
 '2022-04-13',
 '2022-04-11',
 '2022-04-06',
 '2022-04-04',
 '2022-03-30',
 '2022-03-28',
 '2022-03-23',
 '2022-03-21',
 '2022-03-16',
 '2022-03-14',
 '2022-03-11',
 '2022-03-09',
 '2022-03-07',
 '2022-03-02',
 '2022-02-23',
 '2022-02-21',
 '2022-02-16',
 '2022-02-14',
 '2022-02-09',
 '2022-02-07',
 '2022-02-02',
 '2022-02-01',
 '2022-01-31',
 '2022-01-26',
 '2022-01-24',
 '2022-01-19',
 '2022-01-17',
 '2022-01-12',
 '2022-01-10',
 '2022-01-06',
 '2022-01-05',
 '2022-01-04',
 '2022-01-03',
 '2021-12-22',
 '2021-12-20',
 '2021-12-15',
 '2021-12-13',
 '2021-12-09',
 '2021-12-08',
 '2021-12-06',
 '2021-12-02',
 '2021-12-01',
 '2021-11-29',
 '2021-11-24',
 '2021-11-23',
 '2021-11-22',
 '2021-11-18',
 '2021-11-17',
 '2021-11-15',
 '2021-11-11',
 '2021-11-10',
 '2021-11-08',
 '2021-11-04',
 '2021-11-

In [16]:
len(epi_rel_date)

1509

In [17]:
len(show_url)

1509

In [18]:
len(epi_desc)

1509

In [19]:
len(epi_duration)

1509

In [20]:
df = pd.DataFrame({"Description": epi_desc, "Date": epi_rel_date, "Duration": epi_duration, "Url": show_url})

In [21]:
df

,Description,Date,Duration,Url
0,Seth Green's NFTs got stolen because that's wh...,2022-05-25,3957983,https://api.spotify.com/v1/shows/6GWLSnyJKGMDI...
1,"In the aftermath of H Week, we had no choice b...",2022-05-23,4044000,https://api.spotify.com/v1/shows/6GWLSnyJKGMDI...
2,H Week rolls on which left us no choice but to...,2022-05-18,4459128,https://api.spotify.com/v1/shows/6GWLSnyJKGMDI...
3,Did Dave get in a drive-thru altercation? Yes....,2022-05-16,4211880,https://api.spotify.com/v1/shows/6GWLSnyJKGMDI...
4,"Another Friday, another pot of coffee, another...",2022-05-13,3860711,https://api.spotify.com/v1/shows/6GWLSnyJKGMDI...
...,...,...,...,...
1504,the boys are back on the sauce so you know thi...,2019-02-05,2038073,https://api.spotify.com/v1/shows/20REXE78U9IC7...
1505,the squadron sits down on a snowy january even...,2019-01-28,2702054,https://api.spotify.com/v1/shows/20REXE78U9IC7...
1506,brad and john talk soulja boy and his influenc...,2019-01-25,1140062,https://api.spotify.com/v1/shows/20REXE78U9IC7...
1507,brad and john discuss their childhoods and tou...,2019-01-21,2474057,https://api.spotify.com/v1/shows/20REXE78U9IC7...


In [22]:
df["Url"][500]

'https://api.spotify.com/v1/shows/2v9N3b9tVvkX06NAYwzrvU/episodes?offset=0&limit=50&market=US'

In [23]:
show_ids

['6GWLSnyJKGMDIWsYC0RBG2',
 '5nVfEJ9bPUvqN2ml5sUGG2',
 '2v9N3b9tVvkX06NAYwzrvU',
 '0UgblNURXpPYzoBrPHJyi1',
 '7sH9LDMFdARFSHcZGE3R34',
 '44Cx2rZvJZvKnUlC4AU2lK',
 '7L8xnywAsJjuQcSk8PbDRc',
 '20REXE78U9IC7svf8dTrHU']

In [24]:
urls = df["Url"].unique()
    

In [25]:
url_shows = {}

for url in urls:
    for show_id in show_dict:
        if show_id in url:
            url_shows[url] = show_dict[show_id]

In [26]:
url_shows

{'https://api.spotify.com/v1/shows/6GWLSnyJKGMDIWsYC0RBG2/episodes?offset=0&limit=50&market=US': 'Circling Back',
 'https://api.spotify.com/v1/shows/5nVfEJ9bPUvqN2ml5sUGG2/episodes?offset=0&limit=50&market=US': 'Too Much Dip',
 'https://api.spotify.com/v1/shows/2v9N3b9tVvkX06NAYwzrvU/episodes?offset=0&limit=50&market=US': 'Mail-In',
 'https://api.spotify.com/v1/shows/0UgblNURXpPYzoBrPHJyi1/episodes?offset=0&limit=50&market=US': 'Sunday Scaries',
 'https://api.spotify.com/v1/shows/7sH9LDMFdARFSHcZGE3R34/episodes?offset=0&limit=50&market=US': 'Club Cool',
 'https://api.spotify.com/v1/shows/44Cx2rZvJZvKnUlC4AU2lK/episodes?offset=0&limit=50&market=US': 'Brunch',
 'https://api.spotify.com/v1/shows/7L8xnywAsJjuQcSk8PbDRc/episodes?offset=0&limit=50&market=US': 'Outgoing Without Going Out',
 'https://api.spotify.com/v1/shows/20REXE78U9IC7svf8dTrHU/episodes?offset=0&limit=50&market=US': 'Internet Party'}

In [27]:
df["Url"] = df["Url"].replace(url_shows)

In [29]:
df.rename(columns = {"Url" : "Name"}, inplace = True)

In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1509 entries, 0 to 1508
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Description  1509 non-null   object
 1   Date         1509 non-null   object
 2   Duration     1509 non-null   int64 
 3   Name         1509 non-null   object
dtypes: int64(1), object(3)
memory usage: 47.3+ KB


In [31]:
df["Duration"] = df["Duration"]/ 60000

In [32]:
df

,Description,Date,Duration,Name
0,Seth Green's NFTs got stolen because that's wh...,2022-05-25,65.966383,Circling Back
1,"In the aftermath of H Week, we had no choice b...",2022-05-23,67.400000,Circling Back
2,H Week rolls on which left us no choice but to...,2022-05-18,74.318800,Circling Back
3,Did Dave get in a drive-thru altercation? Yes....,2022-05-16,70.198000,Circling Back
4,"Another Friday, another pot of coffee, another...",2022-05-13,64.345183,Circling Back
...,...,...,...,...
1504,the boys are back on the sauce so you know thi...,2019-02-05,33.967883,Internet Party
1505,the squadron sits down on a snowy january even...,2019-01-28,45.034233,Internet Party
1506,brad and john talk soulja boy and his influenc...,2019-01-25,19.001033,Internet Party
1507,brad and john discuss their childhoods and tou...,2019-01-21,41.234283,Internet Party


In [33]:
df["Date"] = pd.to_datetime(df["Date"])

In [34]:
df["Day"] = df["Date"].dt.day_name()

In [35]:
df

,Description,Date,Duration,Name,Day
0,Seth Green's NFTs got stolen because that's wh...,2022-05-25,65.966383,Circling Back,Wednesday
1,"In the aftermath of H Week, we had no choice b...",2022-05-23,67.400000,Circling Back,Monday
2,H Week rolls on which left us no choice but to...,2022-05-18,74.318800,Circling Back,Wednesday
3,Did Dave get in a drive-thru altercation? Yes....,2022-05-16,70.198000,Circling Back,Monday
4,"Another Friday, another pot of coffee, another...",2022-05-13,64.345183,Circling Back,Friday
...,...,...,...,...,...
1504,the boys are back on the sauce so you know thi...,2019-02-05,33.967883,Internet Party,Tuesday
1505,the squadron sits down on a snowy january even...,2019-01-28,45.034233,Internet Party,Monday
1506,brad and john talk soulja boy and his influenc...,2019-01-25,19.001033,Internet Party,Friday
1507,brad and john discuss their childhoods and tou...,2019-01-21,41.234283,Internet Party,Monday


In [36]:
df["Name"].value_counts()

Circling Back                 300
Brunch                        300
Sunday Scaries                216
Club Cool                     165
Mail-In                       161
Too Much Dip                  135
Outgoing Without Going Out    124
Internet Party                108
Name: Name, dtype: int64

In [37]:
import csv 
import sqlite3

connection = sqlite3.connect(r"C:\Users\Haddon\washedmedia.db")

cursor = connection.cursor()

drop_table = "DROP TABLE IF EXISTS washed_media_episodes"

create_table =   '''CREATE TABLE IF NOT EXISTS washed_media_episodes(
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    description TEXT,
                    date DATETIME,
                    duration FLOAT,
                    name TEXT,
                    day TEXT)
                    '''

cursor.execute(drop_table)

cursor.execute(create_table)

df.to_sql('washed_media_episodes', connection, if_exists='replace', index = False)
